In [1]:
from client import start_client
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib.pyplot import figure
from main_func import main,real_partial
from plotting import plot
from jupyterthemes import jtplot

client,w=start_client(5)
client

<Client: 'tcp://127.0.0.1:41348' processes=5 threads=5, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:41348 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 33.69 GB


### Main function
- make_classification parameters: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html?highlight=make_classification#sklearn.datasets.make_classification
- choose e, and chunk size
- run multiple times to produce plots


### Initialize
***

In [2]:
jtplot.style(theme='grade3') #light plot theme

batches=1000 #number of minibatches
batch_size=1 #size of each minibatch
e=0.01 #tolerance for φ(χ)

#compute samples size
training_s=batches*batch_size
n_samples=int(3*training_s/2) # 1/3 of the samples will be used for testing

#parameters for make classification
dataset_params={
    "n_samples":n_samples,
    "n_features":10,
    "n_informative":10,
    "n_redundant":0,
    "n_repeated":0,
    "n_classes":2,
    "n_clusters_per_class":1,
    "flip_y":0,
    "weights":[0.5,0.5],
    "random_state":17
}
threshold=0.03

chunks=100
e_array=[0.01,0.1,1]
minibaches=[5,10,50]
new="yes" #choose if you want to recreate the dataset

In [ ]:
Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,threshold,chunks,minibaches[0])


In [ ]:
E_real1,Acc_real1=real_partial(100)
len_real1=np.array([i for i in range(len(Acc_real1))])

In [ ]:
from plotting import plot
plot()

### Dataset with no noise flip_y=0
***
#### Balanced
- weights=[0.5,0,5]

In [ ]:
# 3 different e
new="yes"
dataset_params["weights"]=[0.5,0.5]
Acc_threshold=[]
Rounds_threshold=[]
Time_threshold=[]

for i in e_array :
    e=i
    Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibaches[0])
    new="no"
    Acc_threshold.append(Acc)
    Rounds_threshold.append(n_rounds)
    Time_threshold.append(time_stamps)
E_real1,Acc_real1=real_partial(100)
len_real1=np.array([i for i in range(len(Acc_real1))])

labels=[[]]
l1=[[]]
l2=[[]]
labels[0]="Real"
print(len(Acc_threshold[0]),Rounds_threshold,len(Time_threshold[0]))
l1[0]=len_real1
l2[0]=Acc_real1
plot(l1,l2,Acc_threshold,Rounds_threshold,Time_threshold,e_array,labels)


Start with num of chunks: 100 and e: 0.01
A new dataset has been created...
Size of chunk:  10
Number of chunks for each worker:  25
In progress...
coo ([array([-11.11036533,   0.52981115,  15.19742297,  -2.65192924,
         9.70030056,  -8.08359072, -16.46385083, -20.11943749,
        -8.25888106, -39.03715197]), 4.992517447666968], 1, 1, 4)
Accuracy: 87.000000
coo ([array([-16.39700812,  -6.33144996,  14.20742982,  11.85960995,
         3.60449123,   4.43258744, -20.11051345, -16.30767979,
       -15.93722942, -39.7346263 ]), 2.5024776070295176], 2, 1, 4)
Accuracy: 89.600000
coo ([array([ -9.1572173 ,  -3.94762364,   8.80901702,  -1.55263822,
         8.34240938,  15.63311247, -33.04586298, -14.7653116 ,
       -25.3107491 , -25.75979625]), 2.500009792269616], 3, 1, 4)
Accuracy: 95.200000
coo ([array([-11.20950131, -13.24780977,  16.07954023,  -3.00651808,
        12.60127984,   5.43675624, -30.36793893, -18.5804144 ,
       -14.71124728, -31.32548604]), 4.958220215081809], 4, 1, 4)

In [ ]:
E_real1,Acc_real1=real_partial(100)
len_real1=np.array([i for i in range(len(Acc_real1))])


#### Unbalanced
***
- weights=[0.9,0.1]

In [ ]:
# 3 different e
new="yes"
dataset_params["weights"]=[0.9,0.1]
UnBal_acc_e=[]
UnBal_rounds_e=[]
unbal_e_time=[]
for i in e_array:
    e=i
    start_time = time.time()
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,100,e,"Unbalanced")
    end_time = time.time()
    unbal_e_time.append(end_time-start_time)
    new="no"
    UnBal_acc_e.append(Acc)
    UnBal_rounds_e.append(Rounds)
# 3 different bach_sizes
UnBal_acc_b=[]
UnBal_rounds_b=[]
unbal_b_time=[]
for i in minibaches:
    e=0.01
    batches=i
    start_time = time.time()
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,batches,e,"Unbalanced")
    end_time = time.time()
    unbal_b_time.append(end_time-start_time)
    UnBal_acc_b.append(Acc)
    UnBal_rounds_b.append(Rounds)

#### Real

In [ ]:
E_real2,Acc_real2=real_partial(100)
len_real2=np.array([i for i in range(len(Acc_real2))])

### Plots
***
- No noise

In [ ]:
#1.1.a balanced acc-rounds /e (multiple lines one for each e)
figure(figsize=(15,15)) 
plt.subplot(221)
plt.plot(Bal_rounds_e[0], Bal_acc_e[0],label=e_array[0],marker='o')
plt.plot(Bal_rounds_e[1], Bal_acc_e[1],label=e_array[1],marker='o')
plt.plot(Bal_rounds_e[2], Bal_acc_e[2],label=e_array[2],marker='o')
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.1.a Accuracy/Rounds for different e \n in a no noise balanced dataset')
plt.legend();

plt.subplot(222)
plt.plot(Bal_rounds_b[0], Bal_acc_b[0],label=minibaches[0],marker='o')
plt.plot(Bal_rounds_b[1], Bal_acc_b[1],label=minibaches[1],marker='o')
plt.plot(Bal_rounds_b[2], Bal_acc_b[2],label=minibaches[2],marker='o')
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.1.b Accuracy/Rounds for different number of minibatches \n in a no noise balanced dataset')
plt.legend();

#1.2.a unbalanced acc-rounds (multiple lines one for each e)
plt.subplot(223)
plt.plot(UnBal_rounds_e[0], UnBal_acc_e[0],label=e_array[0],marker='o')
plt.plot(UnBal_rounds_e[1], UnBal_acc_e[1],label=e_array[1],marker='o')
plt.plot(UnBal_rounds_e[2], UnBal_acc_e[2],label=e_array[2],marker='o')
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.2.a Accuracy/Rounds for different e \n in a no noise Unbalanced dataset')
plt.legend();

#1.2.b unbalanced acc-rounds /baches (multiple lines)
plt.subplot(224)
plt.plot(UnBal_rounds_b[0], UnBal_acc_b[0],label=minibaches[0],marker='o')
plt.plot(UnBal_rounds_b[1], UnBal_acc_b[1],label=minibaches[1],marker='o')
plt.plot(UnBal_rounds_b[2], UnBal_acc_b[2],label=minibaches[2],marker='o')
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.2.b Accuracy/Rounds for different number of minibatches \n in a no noise Unbalanced dataset')
plt.legend();
name="Plots/Exper_NoNoise_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_minibaches_"+str(minibaches)+"_e_"+str(e_array)+"_a.png"
plt.savefig(name)
plt.show();


### Plot centralized 

In [ ]:

plt.plot(len_real1,Acc_real1,label="balanced")
plt.plot(len_real2,Acc_real2,label="unbalanced")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('Centralized behaviour')
plt.legend();
name="Plots/Exper_NoNoise_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_minibaches_"+str(minibaches)+"_e_"+str(e_array)+"_b.png"
plt.savefig(name)
plt.show();


### Plot time 

In [ ]:

figure(figsize=(12,5))

plt.subplot(121)
plt.plot(e_array,[x / 60 for x in bal_e_time],label="balanced",marker='D')
plt.plot(e_array,[x / 60 for x in unbal_e_time],label="unbalanced",marker='o')
plt.ylabel("time(m)")
plt.xlabel("e")
plt.title('Time for different e') 
plt.legend();

plt.subplot(122)
plt.plot(minibaches,[x / 60 for x in bal_b_time],label="balanced",marker='D')
plt.plot(minibaches,[x / 60 for x in bal_b_time],label="unbalanced",marker='o')
plt.ylabel("time(m)")
plt.xlabel("Num of minibaches")
plt.title('Time for different minibaches')
plt.legend();
name="Plots/Exper_NoNoise_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_minibaches_"+str(minibaches)+"_e_"+str(e_array)+"_c.png"
plt.savefig(name)
plt.show();


### Save results

In [ ]:
f = open("result.txt", "a")
f.writelines(["\n\n\nNEW EXPERIMENT:\nparams:",str(dataset_params),"\nminibaches_size:"
              ,str(minibaches),"\ndifferent e's:",str(e_array),"\nn_workers: ",str(len(w))])
f.writelines(["\n\nRESULTS:\nBal_rounds_e",str(Bal_rounds_e),"\nBal_acc_e",str(Bal_acc_e),
            "\nBal_rounds_b",str(Bal_rounds_b),"\nBal_acc_b",str(Bal_acc_b),"\nUnBal_rounds_e", 
              str(UnBal_rounds_e),"\nUnBal_acc_e",str(UnBal_acc_e),"\nUnBal_rounds_b",
              str(UnBal_rounds_b),"\nBal_acc_b",str(UnBal_acc_b),"\nbal_e_time",str(bal_e_time),
              "\nbal_b_time",str(bal_b_time),"\nunbal_e_time",str(unbal_e_time),
              "\nunbal_b_time",str(unbal_b_time),"\nlen_real1",str(len_real1),
              "\nAcc_real1",str(Acc_real1),"\nlen_real2",str(len_real2),
              "\nAcc_real2",str(Acc_real2)])
f.close()

### Shutdown client

In [ ]:
client.close()
client.shutdown()